
1) Посчитать для каждого района Москвы количество точек доступа в этом районе. <br>
2) Найти ближайшую к школе Летово точку доступа городского Wi-Fi.
<br>
<br>
**Входные данные:** <br>csv файл с кодировкой Windows-1251, содержащий в себе информацию о местах расположения точек бесплатной городской сети Wi-Fi.
<br>
<br>
**Используемые библиотеки:** <br>pandas
<br>re
<br>math
<br>
<br>
Импортируем библиотеки

In [1]:
import pandas as pd
import re
from math import radians, cos, sin, asin, sqrt

Считаем файл с данными, который мы предварительно сохранили. Заметим, что вместо запятых, характерных для файлов "csv", там используются ";". Также кодировка этого файла "Windows-1251". Учтем это при считывании. Запишем его в переменную df:

In [2]:
df = pd.read_csv("data-9776-2021-06-17.csv", encoding='Windows-1251', sep = ";")

Выведем основную информацию о данных:

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3059 entries, 0 to 3058
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Latitude_WGS84        3059 non-null   float64
 1   Name                  3059 non-null   object 
 2   AdmArea               3059 non-null   object 
 3   District              3059 non-null   object 
 4   Location              3059 non-null   object 
 5   NumberOfAccessPoints  3059 non-null   int64  
 6   WiFiName              3059 non-null   object 
 7   CoverageArea          3059 non-null   int64  
 8   FunctionFlag          3059 non-null   object 
 9   AccessFlag            3059 non-null   object 
 10  Password              0 non-null      float64
 11  Longitude_WGS84       3059 non-null   float64
 12  global_id             3059 non-null   int64  
 13  geodata_center        3059 non-null   object 
 14  geoarea               0 non-null      float64
 15  Unnamed: 15          

Заметим, что данные представляют из себя таблицу в 16 столбцов и 3059 строчек. Количество строчек небольшое, но будем работать с тем, что имеем.

In [4]:
df.columns 

Index(['Latitude_WGS84', 'Name', 'AdmArea', 'District', 'Location',
       'NumberOfAccessPoints', 'WiFiName', 'CoverageArea', 'FunctionFlag',
       'AccessFlag', 'Password', 'Longitude_WGS84', 'global_id',
       'geodata_center', 'geoarea', 'Unnamed: 15'],
      dtype='object')

Заметим, что названия колонок написаны не в стиле **snake_case**. Исправим это. 

In [5]:
df.columns = df.columns.str.replace(r'(?<!^)(?=[A-Z])', "_").str.lower()


/var/folders/rt/mvnmln8j70d9fmnh5b6pl1qr0000gn/T/ipykernel_41337/2030895304.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace(r'(?<!^)(?=[A-Z])', "_").str.lower()


Заметим, что есть ненужные для анализа столбцы. Так например "password", "geoarea", "unnamed: 15", "latitude_wgs84", "name", "adm_area", "wi_fi_name", "coverage_area", "function_flag", "access_flag", "longitude_wgs84", "global_id", "geodata_center" и т.п. Удалим их

In [6]:
df = df.drop(["name", "adm_area", "wi_fi_name", "coverage_area", "function_flag", "access_flag", "password", "global_id", "geodata_center", "geoarea", "unnamed: 15" ], axis=1)

Посмотрим на получившуюся базу данных

In [7]:
df.head(20)

,latitude__w_g_s84,district,location,number_of_access_points,longitude__w_g_s84
0,55.769309,Басманный район,"город Москва, Новая Басманная улица, дом 4, ст...",1,37.653085
1,55.769692,Красносельский район,"город Москва, Новая Басманная улица, дом 7",2,37.654458
2,55.765217,Басманный район,"город Москва, Старая Басманная улица, дом 13, ...",2,37.659378
3,55.767529,Басманный район,"город Москва, Старая Басманная улица, дом 22",1,37.666265
4,55.767096,Басманный район,"город Москва, Старая Басманная улица, дом 23/9...",2,37.664447
5,55.768621,Басманный район,"город Москва, Старая Басманная улица, дом 33",2,37.667455
6,55.768976,Басманный район,"город Москва, Старая Басманная улица, дом 34",2,37.668718
7,55.769270,Басманный район,"город Москва, Старая Басманная улица, дом 38/2...",1,37.669298
8,55.773183,Басманный район,"город Москва, Бауманская улица, дом 33/2, стро...",2,37.678905
9,55.772561,Басманный район,"город Москва, Бауманская улица, дом 33/2, стро...",1,37.678719


Проверим на дубликаты

In [8]:
df.duplicated().sum()

0

Дубликатов нет. 
## Посчитайте для каждого района Москвы количество точек доступа в этом районе.
Создадим сводную таблицу, где для каждого района посчитаем сумму доступных точек wi-fi

In [9]:
df1 = pd.pivot_table(df, values='number_of_access_points', index='district',
 aggfunc='sum')

Выведем ее

In [10]:
df1.head(10)

,number_of_access_points
district,
Басманный район,353
Бутырский район,3
Гагаринский район,4
Даниловский район,203
Донской район,101
Красносельский район,180
Мещанский район,265
Нижегородский район,4
Пресненский район,507


Отсортируем таблицу по количеству точек доступа

In [11]:
df1.sort_values('number_of_access_points')

,number_of_access_points
district,
район Проспект Вернадского,2
Бутырский район,3
Хорошёвский район,4
Гагаринский район,4
Нижегородский район,4
Южнопортовый район,6
район Марьино,10
район Лефортово,12
район Марьина Роща,37


## Найдите ближайшую к школе Летово точку доступа городского Wi-Fi.
Посчитаем расстояние от школы Летово до каждой точки доступа и выведем адресс ближайшей точки wi-fi. Для этого найдем координаты школы в интернете. Они равны (55.55749672243329, 37.42259192492016)

In [12]:



def calculating(x1, y1, x2, y2):
    r = 6372.8
    d1 = radians(x2 - x1)
    d2 = radians(y2 - y1)
    x1 = radians(x1)
    x2 = radians(x2)
    a = sin(d1/2)**2 + cos(x1)*cos(x2)*sin(d2/2)**2
    c = 2 * asin(sqrt(a))
    return r * c



x1 = 55.55749672243329
y1 = 37.42259192492016


min_distance = 10000000.0

for i, row in df.iterrows():
    x2 = row['latitude__w_g_s84']
    y2 = row['longitude__w_g_s84']
    distance = calculating(x1, y1, x2, y2)
    if distance < min_distance:
        min_distance = distance
        nearest = row

print(nearest['location'], "\n", min_distance, "километров")


город Москва, проспект Вернадского, дом 56 
 13.616868115888428 километров
